In [5]:
# clements.ipynb

# 1. Import libraries
import ibis
import altair as alt
import pandas as pd
import vegafusion as vf

# Enable VegaFusion to handle larger datasets
alt.data_transformers.enable("vegafusion")

# 2. Connect to the database (relative path "data/flights_filtered.db")
conn = ibis.sqlite.connect("data/flights_filtered.db")

# 3. Check what tables are in this database
print("List of tables in the database:", conn.list_tables())

# 4. Pick the relevant table and create a Python object referencing it
flights_table_name = "bos_flights"
flight_data = conn.table(flights_table_name)

# 5. Inspect the table structure by just displaying flight_data
flight_data

# 6. Look at the first few rows without executing (so we see the ibis "Limit" output)
flight_data.head()

# 7. Actually retrieve and display the first few rows as a pandas DataFrame
print(flight_data.head().execute())

# 8. Filter rows where the ORIGIN_AIRPORT is "BOS" and select only ARRIVAL_DELAY and DEPARTURE_DELAY
fd_bos_origin = flight_data.filter(flight_data["ORIGIN_AIRPORT"] == "BOS").execute()

delay_data = fd_bos_origin[["ARRIVAL_DELAY", "DEPARTURE_DELAY"]]

# 9. Inspect the shape of the resulting DataFrame
print("Shape of delay_data:", delay_data.shape)





List of tables in the database: ['bos_flights']
   YEAR  MONTH  DAY  DAY_OF_WEEK ORIGIN_AIRPORT DESTINATION_AIRPORT  DISTANCE  \
0  2015      1    1            4            SJU                 BOS      1674   
1  2015      1    1            4            BOS                 MYR       738   
2  2015      1    1            4            BOS                 MIA      1258   
3  2015      1    1            4            BOS                 BWI       369   
4  2015      1    1            4            ORD                 BOS       867   

   SCHEDULED_DEPARTURE  DEPARTURE_DELAY  SCHEDULED_ARRIVAL  ARRIVAL_DELAY  \
0                  330            -14.0                635          -16.0   
1                  510             -4.0                730          -10.0   
2                  515             -5.0                843            2.0   
3                  540             -4.0                721          -21.0   
4                  540            -11.0                856          -33.0   

  

In [2]:
# 10. Plot Arrival Delay Data as a histogram with Altair
delay_data = delay_data.assign(
    ARRIVAL_DELAY_hr = delay_data["ARRIVAL_DELAY"] / 60
)

arrival_delay_plot = (
    alt.Chart(delay_data).transform_joinaggregate(total="count(ARRIVAL_DELAY_hr)"
    ).transform_calculate(
        pct="1 / datum.total"
    ).mark_bar().encode(
        alt.X("ARRIVAL_DELAY_hr:Q")
        .bin(step=0.25, extent=[-2, 5])
        .title("Delay (hours)"),
        alt.Y("sum(pct):Q")
        .axis(format="%")
        .title("% of Flights")
    ).properties(
        title="Histogram of Arrival Delay (hours) for Flights Departing BOS"
    )
)

arrival_delay_plot

alt.Chart(...)

In [3]:
# 11. Plot Departure Delay Data as a histogram (similar code to arrival delay)
delay_data = delay_data.assign(
    DEPARTURE_DELAY_hr = delay_data["DEPARTURE_DELAY"] / 60
)

departure_delay_plot = (
    alt.Chart(delay_data)
    .transform_joinaggregate(
        total="count(DEPARTURE_DELAY_hr)"
    )
    .transform_calculate(
        pct="1 / datum.total"
    )
    .mark_bar()
    .encode(
        alt.X("DEPARTURE_DELAY_hr:Q")
        .bin(step=0.25, extent=[-2, 5])
        .title("Delay (hours)"),
        alt.Y("sum(pct):Q")
        .axis(format="%")
        .title("% of Flights")
    )
    .properties(
        title="Histogram of Departure Delay (hours) for Flights Departing BOS"
    )
)

departure_delay_plot



alt.Chart(...)

In [4]:
# 12. Write the resulting DataFrame to a CSV file called "delay_data.csv" in the data/ folder
delay_data.to_csv("data/delay_data.csv", index=False)
